In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

tr=pd.read_csv('../input/sjjdbfba/submission.csv')
tr[tr['id']=='99575386e2']
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
tr.to_csv('submission.csv',index=False)

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer

In [ ]:
#引入xlmr-roberta模型
xlmr= AutoModelForSequenceClassification.from_pretrained('joeddav/xlm-roberta-large-xnli')
tokenizer =AutoTokenizer.from_pretrained ('joeddav/xlm-roberta-large-xnli')


In [ ]:
import pandas as pd
train_data=pd.read_csv('../input/contradictory-my-dear-watson/train.csv')
train_data.head(3)

In [ ]:
#kaggle的0，1，2和模型的0，1，2不一样，变换一下
train_data ['label'] =train_data['label'].replace ([0,2], [2, 0])
train_data.head(3)

In [ ]:
train_data_lang=train_data.groupby('language').count().reset_index()[['language','id']]

In [ ]:
#数据集的语言分布
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(10,10))
plt.pie(train_data_lang['id'],labels=train_data_lang['language'],autopct='%1.1f%%')
plt.show()

In [ ]:
def get_tokens_xlmr_model(data):
    batch_tokens=[]
    for i in range(len(data)):
        tokens=tokenizer.encode(data['premise'][i],data['hypothesis'][i],return_tensors='pt',truncation_strategy='only_first')
        batch_tokens.append(tokens)
    return batch_tokens

In [ ]:
def get_predicts_xlmr_model(tokens):
    batch_predicts=[]
    for i in tokens:
        predict=xlmr(i)[0][0]
        predict=int(predict.argmax())
        batch_predicts.append(predict)
    return batch_predicts

In [ ]:
#拿数据集的十分之一跑一遍模型
train_data_lang['sample_count']=train_data_lang['id']/10

In [ ]:
sample_train_data=pd.DataFrame(columns=train_data.columns)

In [ ]:
for i in range (len(train_data_lang)):
    df=train_data[train_data['language']==train_data_lang['language'][i]]
    n=int(train_data_lang['sample_count'][i])
    df=df.sample(n).reset_index(drop=True)
    sample_train_data=sample_train_data.append(df)
sample_train_data=sample_train_data.reset_index(drop=True)
sample_train_data_lang=sample_train_data.groupby('language').count().reset_index()[['language','id']]

In [ ]:
plt.figure(figsize=(10,10))
plt.pie(sample_train_data_lang['id'],labels=sample_train_data_lang['language'],autopct='%1.1f%%')
plt.show()

In [ ]:
sample_train_data_tokens=get_tokens_xlmr_model(sample_train_data)

In [ ]:
sample_train_data_predicts=get_predicts_xlmr_model(sample_train_data_tokens)

In [ ]:
import numpy as np
import seaborn as sns 


In [ ]:
from sklearn.metrics import classification_report
sample_train_data['label']=sample_train_data['label'].astype(str).astype(int)
x=np.array(sample_train_data['label'])
y=np.array(sample_train_data_predicts)
cm=np.zeros((3,3),dtype=int)
np.add.at(cm,[x,y],1)
sns.heatmap(cm,cmap="YlGnBu", annot=True, annot_kws={'size':16},fmt='g',xticklabels=['contradiction', 'neutral', 'entailment'], yticklabels=['contradiction', 'neutral','entailment'])
matrix = classification_report(x,y,labels=[0, 1,2], target_names=
['contradiction','neutral','entailment'])
print('Classification report:\n',matrix)

In [ ]:
#跑出来的的得分
sample_train_data['prediction']=sample_train_data_predicts
sample_train_data['true_prediction']=np.where(sample_train_data['label']==sample_train_data['prediction'],1,0)
sample_train_data_predicted_lang=sample_train_data.groupby('language').agg({'id':'count','true_prediction':'sum'}).reset_index()[['language','id','true_prediction']]
sample_train_data_predicted_lang['accuracy']=round(sample_train_data_predicted_lang['true_prediction']/sample_train_data_predicted_lang['id'],2)
sample_train_data_predicted_lang=sample_train_data_predicted_lang.sort_values(by=['accuracy'],ascending=False)
sample_train_data_predicted_lang

In [ ]:
#拿测试集跑一遍
test_data=pd.read_csv('../input/contradictory-my-dear-watson/test.csv')

In [ ]:
test_data_tokens=get_tokens_xlmr_model(test_data)
test_data_predicts=get_predicts_xlmr_model(test_data_tokens)

In [ ]:
test_data['prediction']=test_data_predicts
del test_data['premise']
del test_data['hypothesis']
del test_data['lang_abv']
del test_data['language']
test_data[test_data['id']=='4436327e71']
test_data['prediction'] =test_data['prediction'].replace ([0,2], [2, 0])
test_data.to_csv('submission.csv',index=False)

In [ ]:
test_data[test_data['id']=='4436327e71']

In [ ]:
test_data

In [ ]:
test_data.to_csv('submission.csv',index=False)

In [ ]:
#对英语文本用roberta模型跑一下
!pip install regex requests
!pip install omegaconf
!pip install hydra-core


In [ ]:
!pip install pytorch

In [ ]:
import torch
roberta=torch.hub.load('pytorch/fairseq','roberta.large.mnli')
roberta.eval()
test_data=pd.read_csv('../input/asdqwer/test.csv')

In [ ]:
test_data_en=test_data[test_data['language']=='English']
def get_tokens_roberta(data):
    batch_tokens=[]
    for i in range(len(data)):
        tokens=roberta.encode(data['premise'][i],data['hypothesis'][i],return_tensors='pt',truncation_strategy='only_first')
        batch_tokens.append(tokens)
    return batch_tokens

In [ ]:
def get_predicts_roberta(tokens):
    batch_predicts=[]
    for i in range(len(tokens)):
        predict=roberta.predict('mnli',tokens[i]).argmax().item()
        batch_predicts.append(predict)
    return batch_predicts

In [ ]:
#替换英文,submit2是用了roberta模型后的新提交
test_data_en_tokens=get_tokens_roberta(test_data_en)
test_data_en_predictions=get_predicts_roberta(test_data_en_tokens)
test_data_en['prediction']=test_data_en_predictions
test_data[test_data['language']=='English']=test_data_en
del test_data['premise']
del test_data['hypothesis']
del test_data['lang_abv']
del test_data['language']
test_data.to_csv('submit2.csv',index=False)